4) 1D Convolutional Neural Network Model Training - takes the output of #3 as inputs and performs 1D Convolutional Neural Network Model model training. Outputs the best model. 

## Build the Model

Construct the Conv1D neural network model.


In [ ]:
# Build the model
model = Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))
model.add(Dropout(0.2))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

model.add(Conv1D(32, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))

model.add(Flatten())
model.add(Dense(units=16, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=7, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

## Train the Model

Train the model with the training data.


In [ ]:
# Train the model
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, patience=4, min_lr=0.0000001)
history = model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test), callbacks=[rlrp])

## Evaluate the Model

Evaluate the model's performance on the test data.


In [ ]:
# Evaluate the model
print("Accuracy of our model on test data:", model.evaluate(x_test, y_test)[1] * 100, "%")

## Plot Training & Validation Metrics

Visualize the training and validation accuracy and loss over epochs.


In [ ]:
# Plot training & validation accuracy and loss
plt.rcParams.update({'font.size': 12})
epochs = [i for i in range(50)]
fig, ax = plt.subplots(1, 2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']

fig.set_size_inches(20, 6)
ax[0].plot(epochs, train_loss, label='Training Loss', marker='o', linewidth=2)
ax[0].plot(epochs, test_loss, label='Testing Loss', marker='.', linewidth=2)
ax[0].set_title('Training & Testing Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(epochs, train_acc, label='Training Accuracy', marker='o', linewidth=2)
ax[1].plot(epochs, test_acc, label='Testing Accuracy', marker='.', linewidth=2)
ax[1].set_title('Training & Testing Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")

plt.subplots_adjust(wspace=0.3)
plt.show()

## Confusion Matrix and Classification Report

Generate a confusion matrix and classification report to evaluate the model's performance.


In [ ]:
# Predict the labels for test data
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Confusion Matrix
cm = confusion_matrix(y_test_labels, y_pred)
plt.figure(figsize=(12, 10))
cm = pd.DataFrame(cm, index=[i for i in encoder.categories_[0]], columns=[i for i in encoder.categories_[0]])
sns.heatmap(cm, linecolor='white', cmap='Purples', linewidth=1, annot=True, fmt='')
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.show()

# Classification Report
print(classification_report(y_test_labels, y_pred, target_names=[i for i in encoder.categories_[0]]))

## Save the Model

Save the trained model to a file for later use.


In [ ]:
# Save the model to a file
model.save('emotion_recognition_model.h5')
print("Model saved to 'emotion_recognition_model.h5'")